In [1]:
from src.utils import import_mesh, plot_function, plot_vector_field
from src.models.cell_models import ReparametrizedFitzHughNagumo
from src.models.dynamics_models import BidomainModel
from dolfinx.plot import vtk_mesh
import numpy as np
import pyvista
import ufl
from dolfinx import mesh, fem
from mpi4py import MPI

In [2]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
cell_model = ReparametrizedFitzHughNagumo()


class Model(BidomainModel):
    def __init__(
        self,
    ):
        pass

    def initial_V_m(self):
        return lambda x: x[0] ** 2 + x[1] ** 2 < 0.2**2, 0.0

    def ischemia(self):
        return (
            lambda x: (x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2 < 0.5**2,
            model.M_i / 10,
            model.M_e / 10,
        )


model = Model()
model.setup(domain)

## Popravit onaj visualize da radi za ovu step metodu koju san napravia, napisat dokumentacije za RK2 u utilsima i ovaj step

In [3]:
model.plot_ischemia()

Widget(value='<iframe src="http://localhost:46559/index.html?ui=P_0x7fea69ab0590_0&reconnect=auto" class="pyvi…